In [62]:
import re

line = """
https://www.youtube.com/watch?v=wofB1wzyYYI
http://github.com/VikParuchuri/marker/blob/master/tests/renderers/test_markdown_renderer.py
This is a valid text line without a link.
"""
 
url_pattern = re.compile(r"http?s?://[^\s]+")

 
links = []
matches =url_pattern.findall(line)
for match in matches:
 
    links.append({
        "text": match,
        "url": match
    })

for link in links:
    print(link)


{'text': 'https://www.youtube.com/watch?v=wofB1wzyYYI', 'url': 'https://www.youtube.com/watch?v=wofB1wzyYYI'}
{'text': 'http://github.com/VikParuchuri/marker/blob/master/tests/renderers/test_markdown_renderer.py', 'url': 'http://github.com/VikParuchuri/marker/blob/master/tests/renderers/test_markdown_renderer.py'}


In [7]:
import pandas as pd
from pandas import DataFrame
from lxml import html

def extract_tables_from_html(html_content: str) -> list[DataFrame] | None:
    tree = html.fromstring(html_content)
    tables = tree.xpath('//table')

    if not tables:
        return None

    tables_html = [html.tostring(table, method="html").decode() for table in tables]

    print(tables_html)
    dfs:list[DataFrame] = []
    for table_html in tables_html:
        try:
            df = pd.read_html(table_html)[0]
            dfs.append(df)
        except ValueError:
            continue

    return dfs if dfs else None
 
def save_tables_to_markdown(dfs: list[DataFrame], file_name: str) -> None:
    with open(file_name, 'w') as md_file:
        for i, df in enumerate(dfs):
            md_file.write(f"## Table {i + 1}\n\n")
            md_file.write(df.to_markdown())
            md_file.write("\n\n") 
 
with open('sample.html', 'r') as f:
    html_content = f.read()

dfs = extract_tables_from_html(html_content)

if dfs is not None:
    save_tables_to_markdown(dfs, 'tables.md')



In [26]:
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
import io

def extract_tables_from_html(html_content: str) -> list[DataFrame] | None:
    soup = BeautifulSoup(html_content, "html.parser")
    tables = soup.find_all("table")
    if not tables:
        print("There is no table")
        return None

    tables_html = [str(table) for table in tables]

    dfs:list[DataFrame] = []
    for table_html in tables_html:
        try:
            str_buffer = io.StringIO(table_html)
            df = pd.read_html(str_buffer)[0]
            dfs.append(df)
        except ValueError:
            print("There is no table")
            continue
    if dfs:
        return dfs
    return None

def save_tables_to_markdown(dfs: list[DataFrame], file_name: str) -> None:
    with open(file_name, "w") as md_file:
        for i, df in enumerate(dfs):
            md_file.write(f"## Table {i + 1}\n\n")
            md_file.write(df.to_markdown())
            md_file.write("\n\n")

with open("sample.html") as f:
    html_content = f.read()

dfs = extract_tables_from_html(html_content)
if dfs is not None:
    save_tables_to_markdown(dfs, "tables.md")


yes


In [ ]:
import openpyxl
from openpyxl.drawing.image import Image
from PIL import Image as PILImage

pxl_doc = openpyxl.load_workbook('DimOrder.xlsx')
sheet = pxl_doc['ORDER_DETAILS']
 
for i, image in enumerate(sheet._images):
  
    if isinstance(image, Image):
     
        img_data = image.ref  
    
        pil_img = PILImage.open(img_data)
 
        img_filename = f"image_{i}.png"  
        pil_img.save(img_filename)   
        print(f"Saved image: {img_filename}")


Saved image: image_0.png


In [45]:
sheet_index = ["1", "Order", "3", "5"]

parsed_list = [int(item) if item.isdigit() else item for item in sheet_index]

print(parsed_list)



[1, 'Order', 3, 5]


In [ ]:
import openpyxl
from openpyxl.drawing.image import Image
from PIL import Image as PILImage
 
pxl_doc = openpyxl.load_workbook('DimOrder.xlsx')
 
for sheet_name in pxl_doc.sheetnames:
    sheet = pxl_doc[sheet_name]
 
    for i, image in enumerate(sheet._images):
     
        if isinstance(image, Image):
            img_data = image.ref   
         
            pil_img = PILImage.open(img_data)
 
            img_filename = f"{sheet_name}_image_{i}.png"   
            pil_img.save(img_filename)
            print(f"Saved image from {sheet_name}: {img_filename}")


Saved image from ORDER_DETAILS: ORDER_DETAILS_image_0.png
Saved image from customer: customer_image_0.png
Saved image from vendor: vendor_image_0.png


In [ ]:
from s3fs import S3FileSystem
from uuid import uuid4   

s3fs = S3FileSystem(
    endpoint_url= ENDPOINT_URL,
    key=MINIO_ROOT_USER,
    secret=MINIO_ROOT_PASSWORD,
    use_ssl=False,
)

def save_file_s3(s3fs: S3FileSystem, file_name: str, content: bytes | str) -> str:
    new_uuid = uuid4()
    s3_url = f"{BUCKET_NAME}/{new_uuid}_{file_name}"
    if isinstance(content, str):
        content = content.encode(encoding="utf-8", errors="ignore")
    with s3fs.open(s3_url, mode="wb") as f:
        f.write(content)
        return s3_url

def save_img_s3(s3fs: S3FileSystem, image_name:str, image:"Image") -> str:
    image_name = image_name.lower()
    buffered = io.BytesIO()
    image.save(buffered, format=image_name.split(".")[-1])
    img_b = buffered.getvalue()
    return save_file_s3(s3fs, image_name, img_b)

In [ ]:
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from PIL import Image as PILImage
from s3fs import S3FileSystem
import io 

def extract_excel_images(s3fs: S3FileSystem, excel_content: bytes) -> dict[str, str]:
    """
    Extract images from an Excel file and store them in S3.
    return a dictionary mapping image names to S3 paths.
    """
    pxl_doc = load_workbook(filename=io.BytesIO(excel_content))
    all_images = {}
    
    for sheet_name in pxl_doc.sheetnames:
        sheet = pxl_doc[sheet_name]
        
        for i, image in enumerate(sheet._images):
            if isinstance(image, Image):
                img_data = image.ref
                pil_image = PILImage.open(img_data)

                img_name = f"{sheet_name}_image_{i}.png"
                saved_img_path = save_img_s3(s3fs, img_name, pil_image)
                
                all_images[img_name] = saved_img_path

    return all_images

xlsx_filepath = "DimOrder.xlsx"


with open(xlsx_filepath, mode="rb") as f:
    content = f.read()
    images = extract_excel_images(s3fs, content)
    print(images)

![_page_0_Picture_0.png](_page_0_Picture_0.png)

A UnitedHealthcare Company

![_page_0_Picture_2.png](_page_0_Picture_2.png)

#### Peoples Health Choices 65 HMO-POS H1961-014-001

#### Peoples Health Medicare Advantage LA-0004 нмо-роѕ H1961-020-000

These HMO-POS plans are available in Ascension, East Baton Rouge, East Feliciana, Iberville, Jefferson, Livingston, Orleans, St. Charles, St. Helena and West Baton Rouge parishes.

#### Peoples Health Choices PPO H4544-001

#### Peoples Health Patriot PPO (MA-Only) H4544-002

These PPO plans are available statewide.

## Peoples Health

Founded in by medical providers and physicians in Louisiana

Focused on the needs of people with

Medicare

Peoples Health Choices 65 enrolled its first members in

and has consistently earned high ratings from Medicare.

We offer three Medicare plan types:

Health Maintenance Organization (HMO-POS*) Preferred Provider Organization (PPO) Dual-Eligible Special Needs Plan (HMO-POS* D-SNP)

* Point-of-service (P